In [1]:
import math
import numpy as np
import cmath 



In [12]:
fo = 10.7e6
wo = 2*np.pi*fo
# Qo de los transformadores
Qo = 100

# MMBT918 Ic = 8mA, VCE=10V, fo=10.7MHz
y11 =   1.80e-3  + 0.85e-3*1j
y12 = - 0.01e-3  - 0.08e-3*1j
y21 =    110e-3  -   20e-3*1j 
y22 =   0.25e-3  + 0.12e-3*1j

ytr = np.matrix([[y11, y12],[ y21, y22]])
# MMBT918 Ic = 4mA, VCE=10V, fo=10.7MHz
y11 =   1.30e-3  + 0.80e-3*1j
y12 = - 0.01e-3  - 0.08e-3*1j
y21 =     95e-3  -   19e-3*1j 
y22 =   0.08e-3  + 0.20e-3*1j




In [13]:
def printY(y):
    print( 'y11= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[0,0]) )
    print( 'y12= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[0,1]) )
    print( 'y21= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[1,0]) )
    print( 'y22= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[1,1]) )

def printYp(y):
    print('Polar')
    print( '|y11|= {0:6.4E}, <y11= {1:6.1F}' .format(abs(y[0,0]), cmath.phase(ytr[0,0])*180./np.pi) )
    print( '|y12|= {0:6.4E}, <y12= {1:6.1F}' .format(abs(y[0,1]), cmath.phase(ytr[0,1])*180./np.pi) )
    print( '|y21|= {0:6.4E}, <y21= {1:6.1F}' .format(abs(y[1,0]), cmath.phase(ytr[1,0])*180./np.pi) )
    print( '|y22|= {0:6.4E}, <y22= {1:6.1F}' .format(abs(y[1,1]), cmath.phase(ytr[1,1])*180./np.pi) )

    
def getSpice(y):
    print( '.param g11= {0.real:6.4}' .format(y[0,0]) )
    print( '.param g12= {0.real:6.4E}' .format(y[0,1]) )
    print( '.param g21= {0.real:6.4E}' .format(y[1,0]) )
    print( '.param g22= {0.real:6.4E}' .format(y[1,1]) )
    print( '.param c11= {0:6.4E}' .format(y[0,0].imag/(wo)) )
    print( '.param c12= {0:6.4E}' .format((y[0,1].imag)/(wo)) )
    print( '.param c21= {0:6.4E}' .format((y[1,0].imag)/(wo)) )
    print( '.param c22= {0:6.4E}' .format(y[1,1].imag/(wo)) )
    
def printInOut(yg,yl):
    print('######## Entrada #########')
    print( 'rin= {0:4.2E}' .format(1/yg.real) )
    print( 'cin= {0:4.2E}' .format(-yg.imag/wo) )
    print('######## Salida #########')
    print( 'ro= {0:4.2E}' .format(1/yl.real) )
    print( 'co= {0:4.2E}' .format(-yl.imag/wo) )


In [16]:
def linvill(y):
    ya = y[0,1]*y[1,0]
    C = ( abs(ya) + ya.real )/ (2*y[0,0].real*y[1,1].real)
    print('###############  LINVILL   #####################')
    print( 'coeficiente de linvill C= {:4.2F}' .format(C))
    if C <= 1: print( 'si C es menor a uno es incondicionalmente estable')
    else    : print( 'si C es mayor o igula a 1 es potencialmente inestable')
    
    
def stern(y, yg, yl):
    ya = y[0,1]* y[1,0]
    K = (2*(yg.real+y[0,0].real)*(yl.real+y[1,1].real))/( abs(ya) + ya.real )
    print('###############  Stern   #####################')
    print('coeficiente de Stern K=', K)
    print('si k es MAYOR a 1 no oscilara' )   

def Ar(y,yg,yo): # vo/ig
    ya = y[0,1]* y[1,0]
    Ar = -y[1,0]/((y[0,0]+yg)*(y[1,1]+yl)-ya)
    print('###############  Ganancia de transresistencia (vo/ig)   #########')
    print( 'vo/ig = {:4.2F}' .format(abs(Ar)))
    print( 'vo/ig = {:4.2F} dB' .format(20*np.log10(abs(Ar)))) 
    
def At(y,yg,yl) :  
    ya = y[0,1]* y[1,0]
    pgain = 4*yg.real*yl.real*abs(y[1,0])**2 / (abs((yl+y[1,1])*(y[0,0]+yg)-ya))**2
    print('###############  Ganancia de potencia   #####################')
    print( 'Power Gain = {:4.2F}' .format(pgain))
    print( 'Power Gain = {:4.2F} dB' .format(10*np.log10(pgain)))    
    
    
def neutraliza(y,wo,n = 1):
    Cn = -y[0,1].imag/wo*n
    y11 =  wo*Cn*1j
    y12 =  wo*Cn*1j/n
    y21 =  wo*Cn*1j/n
    y22 =  wo*Cn*1j/n**2
    yn  = np.matrix([[y11, y12],[ y21, y22]])
    # cuadripolo resultante
    yt  = y + yn
    print('cap=',Cn, 'n=', n)
    return (yt,Cn)

def yg_yl(y):
    yl = y[1,1].real - y[1,1].imag*1j
    for i in range(10):
        yin  = y[0,0] - y[0,1]*y[1,0]/(y[1,1]+yl)
        yg   = yin.real - yin.imag*1j
        yout = y[1,1] - y[0,1]*y[0,1]/(y[0,0]+yg)
        yl   = yout.real - yout.imag*1j
    return (yg, yl)


def divC(Q,wo,Rin,RL):
    Qm = ((RL/Rin)*(1+Q**2)-1)**0.5
    C  = Q/Rin/wo
    Cs = C*(1+1/Q**2)
    Cb = Qm/R/wo
    Cbs = Cb*(1+1/Qm**2)
    Ca = 1/(1/Cs-1/Cbs)
    return(Ca,Cb)


def yf(Qc,L,wo,w):
    return (wo*L*Qc)*(1+1j*Qc*(w/wo-w/w))

def yin(y,yl):
    return y[0,0] - (y[0,1]*y[1,0])/(y[1,1]+yl)

def yout(y,yg):
    return y[1,1] - (y[0,1]*y[1,0])/(y[0,0]+yg)

In [19]:
printY(ytr)
print(yin(ytr,1/330))
print(yout(ytr,2/3e3).imag)

# calculo linvill del transistor
linvill(ytr)
# stern 
stern(ytr, 2/3e3, 1/330) 

# admitancia de entrada y de salida para MTE
(yg,yl) = yg_yl(ytr) 
printInOut(yg,yl)

y11= 1.80E-03+j(8.50E-04)
y12= -1.00E-05+j(-8.00E-05)
y21= 1.10E-01+j(-2.00E-02)
y22= 2.50E-04+j(1.20E-04)
(0.00271777380759+0.00343813501822j)
0.00289926953683
###############  LINVILL   #####################
coeficiente de linvill C= 7.02
si C es mayor o igula a 1 es potencialmente inestable
###############  Stern   #####################
coeficiente de Stern K= 2.56305677735
si k es MAYOR a 1 no oscilara
######## Entrada #########
rin= 1.39E+02
cin= 2.68E-10
######## Salida #########
ro= 4.00E+03
co= 1.79E-12


In [8]:
# neutralizador
nf = 6
(yu,Cp) = neutraliza(ytr,wo,nf)

printY(yu)

# calculo linvill del transistor
linvill(yu)

# admitancia de entrada y de salida para MTE
yg = (0.00377534048972-0.000386925950314*1j)
yl = (0.000232328645525-0.000158041596945*1j)
printInOut(yg,yl)

# stern 
stern(yu, yg, yl)


#Ar(yu,yg,yl)
#At(yu,yg,yl)

cap= 7.13966099851e-12 n= 6
y11= 1.30E-03+j(1.28E-03)
y12= -1.00E-05+j(0.00E+00)
y21= 9.50E-02+j(-1.89E-02)
y22= 8.00E-05+j(2.13E-04)
###############  LINVILL   #####################
coeficiente de linvill C= 0.09
si C es menor a uno es incondicionalmente estable
######## Entrada #########
rin= 2.65E+02
cin= 5.76E-12
######## Salida #########
ro= 4.30E+03
co= 2.35E-12
###############  Stern   #####################
coeficiente de Stern K= 169.926888366
si k es MAYOR a 1 no oscilara


Tengo 3 etapas sintonizadas, realizadas con 2 transistores.

1 SS : Qca = 53.5 foa = 10.52679 MHz

2 SS : Qcc = 53.5 fob = 10.87320 MHz

3 SS : Qcb = 26.7 foc = 10.70000 MHz

In [230]:
rg = 1500
rl = 1500

Qo =100

#[(53.500000000000007, 10526794.919243112), (26.75, 10700000.0), (53.500000000000007, 10873205.080756888)]
Qca = 53.5
Qcb = 53.5
Qcc = 26.75
woa = 10526794.91*2*math.pi
wob = 10873205.08*2*math.pi
woc = wo


# dado el transistor
ri = 1/yg.real
bi = -yg.imag
ro = 1/yl.real
bo = -yl.imag


In [238]:
# calculo del segundo sintonizado
Qsb = 1/(1/Qcb-1/Qo)
#ybin = yin(yu,yf())

rextb = ro/2

Lb = (1/Qsb)*rextb/wob 

# transformador
ns  = (ro/ri)**0.5
Lbs = Lb / ns**2
np  = 1 


Cb = 1/(wob**2*Lb)- (bo/ns**2)/wob - (bo/np**2)/wob

print('Lb =',Lb)
print('Lbs=',Lbs)
print('Cb=',Cb)

Lb = 7.95250830402e-07
Lbs= 8.78854242492e-09
Cb= 2.66257336544e-10


In [239]:
# calculo del primer sintonizado
Qsa = 1/(1/Qca-1/Qo)

rexta = ro/2

La = (1/Qsa)*rexta/woa 
# transformador
ns  = (ro/ri)**0.5
Las = La / ns**2
np  = (ro/rg)**0.5
Lap1= La/ np**2
Lap2= La/ (np/(np-1))**2



Ca = 1/(woa**2*La)- (bi/ns**2)/woa

print('Lap1=',Lap1)
print('Lap2=',Lap2)
print('Las=',Las)
print('Ca=',Ca)

Lap1= 9.85563030074e-08
Lap2= 3.50920870361e-07
Las= 9.07775110633e-09
Ca= 2.78263967412e-10


In [240]:
# calculo del tercer sintonizado
Qsc = 1/(1/Qcc-1/Qo)

rextc = ro/2

Lc = (1/Qsc)*rextc/woc 
# transformador
ns  = (ro/rl)**0.5
Lcs = Lc / ns**2
np  = 1



Cc = 1/(woc**2*Lc)- (bi/ns**2)/woc

print('Lc=',Lc)
print('Lcs=',Lcs)
print('Cc=',Cc)

Lc= 2.54602465392e-06
Lcs= 3.05479070176e-07
Cc= 8.67241063725e-11


In [241]:
getSpice(yu)

.param g11= 0.0013
.param g12= -1.0000E-05
.param g21= 9.5000E-02
.param g22= 8.0000E-05
.param c11= 1.9039E-11
.param c12= 0.0000E+00
.param c21= -2.8142E-10
.param c22= 3.1732E-12


In [242]:
# sintonizado entrada
print( '.param Lap1= {0:6.4E}' .format(Lap1))
print( '.param Lap2= {0:6.4E}' .format(Lap2))
print( '.param Las = {0:6.4E}' .format(Las))
print( '.param Ca  = {0:6.4E}' .format(Ca))
print( '.param rpa= {0:6.4E}' .format(Qo*Las*woa))


.param Lap1= 9.8556E-08
.param Lap2= 3.5092E-07
.param Las = 9.0778E-09
.param Ca  = 2.7826E-10
.param rpa= 6.0042E+01


In [243]:
# sintonizado intermedio
print( '.param Lb  = {0:6.4E}' .format(Lb))
print( '.param Lbf = {0:6.4E}' .format(Lb/nf**2))
print( '.param Cp  = {0:6.4E}' .format(Cp))

print( '.param Lbs = {0:6.4E}' .format(Lbs))
print( '.param Cb  = {0:6.4E}' .format(Cb))
print( '.param rpb = {0:6.4E}' .format(Qo*Lbs*wob))


.param Lb  = 7.9525E-07
.param Lbf = 2.2090E-08
.param Cp  = 7.1397E-12
.param Lbs = 8.7885E-09
.param Cb  = 2.6626E-10
.param rpb = 6.0042E+01


In [244]:
# sintonizado tercer
print( '.param Lc  = {0:6.4E}' .format(Lc))
print( '.param Lcf = {0:6.4E}' .format(Lc/nf**2))
print( '.param Lcs = {0:6.4E}' .format(Lcs))
print( '.param Cc  = {0:6.4E}' .format(Cc))
print( '.param rpc = {0:6.4E}' .format(Qo*Lcs*woc))

.param Lc  = 2.5460E-06
.param Lcf = 7.0723E-08
.param Lcs = 3.0548E-07
.param Cc  = 8.6724E-11
.param rpc = 2.0537E+03
